In [1]:
#!pip install -q torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install torch-geometric
#!pip install optuna

In [3]:
!nvidia-smi

Tue Jun 01 17:53:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   51C    P0    64W / 215W |    568MiB /  8192MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [4]:
import torch
import optuna
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
import numpy as np
from modules.model import Net
from sklearn.metrics import f1_score
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
from datetime import datetime
import random
from torch_geometric.data import GraphSAINTNodeSampler
import pickle

In [5]:
from torch_geometric.datasets import Amazon
#dataset = Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())[0]
dataset = Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())[0]
indices=list(range(len(dataset.x)))
train_indices = torch.tensor(indices[:int(0.7*len(indices)+1)])
val_indices = torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
test_indices = torch.tensor(indices[int(0.8*len(indices)+1):])
train_mask = torch.tensor([False]*len(indices))
test_mask = torch.tensor([False]*len(indices))
val_mask = torch.tensor([False]*len(indices))
train_mask[train_indices] =True
test_mask[test_indices]=True
val_mask[val_indices]=True


In [6]:

#data_cora_new
#flag = False
#if flag: 
#    data = dataset

 #   indices_sh=list(range(len(data.x)))
 #   indices=list(range(len(data.x)))
 #   random.shuffle(indices_sh)
 #   x_new_map={}
 #   for i,x in enumerate(indices_sh):
 #       x_new_map[x] = i
 #   data_x_new = data.x[indices_sh]
 #   data_y_new = data.y[indices_sh]
 #   data_edge_index = torch.tensor(list(map(lambda x: [x_new_map[int(x[0])],x_new_map[int(x[1])]] ,data.edge_index.t()))).t()

#    data.x=data_x_new
 #   data.edge_index=data_edge_index
  #  data.y=data_y_new
   # with open('data_photo_new.pickle','wb') as f:
    #    pickle.dump(data,f)
#else:
 #   with open('data_photo_new.pickle','rb') as f:
  #      data = pickle.load(f)

#dataset.x = torch.nn.init.xavier_uniform_(torch.empty(len(data.x), 128 ))


In [7]:

#dataset = torch.load("./data.pt")[0]
#data = dataset
#indices=list(range(len(dataset.x)))
#train_indices = torch.tensor(indices[:int(0.7*len(indices)+1)])
#val_indices = torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
#test_indices = torch.tensor(indices[int(0.8*len(indices)+1):])
#train_mask = torch.tensor([False]*len(indices))
#test_mask = torch.tensor([False]*len(indices))
#val_mask = torch.tensor([False]*len(indices))
#train_mask[train_indices] =True
#test_mask[test_indices]=True
#val_mask[val_indices]=True


**Обучаем на фичах**

In [8]:
def objective(trial):
        # Integer parameter
        с = trial.suggest_categorical("c", [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9, 10,20,30,40,50,60,70,80,90,100])
        # варьируем параметры

        
        out = dataset.x
        y = dataset.y.cpu().detach().numpy()
        #y=self.y.cpu()
        clf = LogisticRegression(max_iter = 3000,C=с).fit(out.cpu().detach()[train_mask].numpy(), y[train_mask])
        accs_micro = []
        accs_macro = []
        #for mask in [train_mask,test_mask,val_mask]:

         #   accs_micro.append(f1_score(y[mask],clf.predict(out.cpu().detach()[mask].numpy()), average='micro'))
          #  accs_macro.append(f1_score(y[mask],clf.predict(out.cpu().detach()[mask].numpy()), average='macro'))
        score = np.sqrt(f1_score(y[val_mask],clf.predict(out.cpu().detach()[val_mask].numpy()), average='micro') *f1_score(y[val_mask],clf.predict(out.cpu().detach()[val_mask].numpy()), average='macro'))       
        return f1_score(y[val_mask],clf.predict(out.cpu().detach()[val_mask].numpy()), average='micro')

    
def run(number_of_trials):

        study = optuna.create_study(direction="maximize")
        study.optimize(objective,n_trials = number_of_trials)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))

In [9]:
def learn(data,train_mask,test_mask,val_mask):    
        out = data.x
        y = data.y.cpu().detach().numpy()
        #y=self.y.cpu()
        clf = LogisticRegression(max_iter = 3000,C=0.4).fit(out.cpu().detach()[train_mask].numpy(), y[train_mask])
        accs_micro = []
        accs_macro = []

        for mask in [train_mask,test_mask,val_mask]:

            accs_micro.append(f1_score(y[mask],clf.predict(out.cpu().detach()[mask].numpy()), average='micro'))
            accs_macro.append(f1_score(y[mask],clf.predict(out.cpu().detach()[mask].numpy()), average='macro'))
                
        return accs_micro,accs_macro

    

**обучаем на свертках**

In [10]:
import pickle
import os

class Main():
    def __init__(self,datasetname,conv, device,loss_function, mode ,dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask):
        data = dataset
        self.Conv = conv
        self.device = device
        self.x = data.x
        self.y = data.y.squeeze()
        self.data=data.to(device)
        self.loss = loss_function
        self.mode = mode
        self.datasetname=datasetname
        self.train_indices =train_indices# torch.tensor(indices[:int(0.7*len(indices)+1)])
        self.val_indices =val_indices# torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
        self.test_indices = test_indices#torch.tensor(indices[int(0.8*len(indices)+1):])
        self.train_mask = train_mask#torch.tensor([False]*len(indices))
        self.test_mask = test_mask#torch.tensor([False]*len(indices))
        self.val_mask =val_mask# torch.tensor([False]*len(indices))
        self.flag = self.loss["flag_tosave"]
        
        super(Main, self).__init__()
    def sampling(self,Sampler,epoch,nodes,loss):
        if (epoch == 0): 
            if self.flag:  
                if "alpha" in self.loss: 
                    name_of_file = self.datasetname+"_samples_"+loss["Name"]+"_alpha_"+str(loss["alpha"])+".pickle"
                elif "betta" in self.loss: 
                    name_of_file = self.datasetname+"_samples_"+loss["Name"]+"_betta_"+str(loss["betta"])+".pickle"
                else:
                    name_of_file = self.datasetname+"_samples_"+loss["Name"]+".pickle"             
                if os.path.exists(name_of_file):
                    with open(name_of_file,'rb') as f:
                        self.samples = pickle.load(f)
                else:
                    self.samples = Sampler.sample(nodes) 
                    with open(name_of_file,'wb') as f:
                        pickle.dump(self.samples,f)
            else:
                
                self.samples = Sampler.sample(nodes)
 
    def train(self, model,data,optimizer,Sampler,train_loader,dropout,epoch,loss):
        model.train()   
        total_loss = 0
        optimizer.zero_grad()
        if model.mode == 'unsupervised':
            if model.conv=='GCN':
                arr=torch.nonzero(self.train_mask == True)
                indices_of_train_data = ([item for sublist in arr for item in sublist])
                out = model.inference(data.to(device),dp=dropout)
                samples = self.sampling(Sampler,epoch, indices_of_train_data,loss)
                loss = model.loss(out[self.train_mask], self.samples)
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    if len(train_loader.sizes) == 1:
                        adjs = [adjs]
                    adjs = [adj.to(device) for adj in adjs]
                    out = model.forward(data.x[n_id.to(device)].to(device), adjs)
                    self.sampling(Sampler,epoch,n_id[:batch_size],loss)                 
                    loss = model.loss(out, self.samples)#pos_batch.to(device), neg_batch.to(device))
                    total_loss+=loss
            total_loss.backward()
            optimizer.step()      
            return total_loss /len(train_loader)
        elif model.mode== 'supervised':
            if model.conv=='GCN':
                out = model.inference(data.to(device),dp=dropout)
                y=self.y.to(self.device)
                loss = model.loss_sup(out[self.train_mask],y[self.train_mask])
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    if len(train_loader.sizes) == 1:
                        adjs = [adjs]
                    adjs = [adj.to(device) for adj in adjs]
                    out = model.forward(data.x[n_id].to(device), adjs)
                    y=self.y.to(self.device)
                    loss = model.loss_sup(out,y[n_id[:batch_size]])
                    total_loss+=loss
            total_loss.backward(retain_graph=True)
            optimizer.step()      
            return total_loss /len(train_loader)       

    @torch.no_grad()
    def test(self, model,data,classifier,c,**kwargs):#,n_estimators,learning_rate_carboost, max_depth): 
        model.eval()
        out = model.inference(data.to(device))
        y_true = self.y.cpu().detach().numpy()
        self.y=self.y.cpu()
        if model.mode == 'supervised':
            y_true = self.y.unsqueeze(-1)
            y_pred = out.cpu().argmax(dim=-1, keepdim=True)

            accs_micro = []
            accs_macro = []
            for mask in [self.train_mask,self.test_mask,self.val_mask]:
                accs_micro += [f1_score(self.y.detach()[mask].cpu().numpy(),y_pred[mask], average='micro')]
                accs_macro += [f1_score(self.y.detach()[mask].cpu().numpy(),y_pred[mask], average='macro')]
                
            return accs_micro,accs_macro
            return accs
        elif model.mode == 'unsupervised': 
            if classifier == 'logistic regression':
                #print(sum(sum(out)))
                clf = LogisticRegression(max_iter = 3000,C=c).fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            else:
                n_estimators= kwargs["n_estimators"]
                learning_rate_catboost = kwargs["learning_rate_catboost"]
                max_depth = kwargs["max_depth"]
                
                clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate = learning_rate_catboost, max_depth=max_depth, random_state=0)
                clf.fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            accs_micro = []
            accs_macro = []
            for mask in [self.train_mask,self.test_mask,self.val_mask]:
                accs_micro += [f1_score(self.y.detach()[mask].cpu().numpy(),clf.predict(out.cpu().detach()[mask].numpy()), average='micro')]
                accs_macro += [f1_score(self.y.detach()[mask].cpu().numpy(),clf.predict(out.cpu().detach()[mask].numpy()), average='macro')]
                
            return accs_micro,accs_macro

    def run(self,hidden_layer=64,out_layer=128,dropout=0.0,size=1,learning_rate=0.001,c=100):
        

        classifier = "logistic regression"
        train_loader = NeighborSampler(self.data.edge_index, node_idx=self.train_mask, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        Sampler = self.loss["Sampler"]
        LossSampler = Sampler(self.datasetname,self.data,device=device,mask=self.train_mask,loss_info=self.loss)
        model = Net(dataset = self.data,mode=self.mode,conv=self.Conv,loss_function=self.loss,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = (size),dropout = dropout)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)
                #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
        scheduler=lr_scheduler.StepLR(optimizer, step_size=25,gamma=0.1)
        losses=[]
        train_accs_mi=[]
        test_accs_mi=[]
        val_accs=[]
        name_of_plot='conv: '+model.conv+', mode: '+model.mode+', loss from '+self.loss["Name"]
        train_accs_ma = []
        test_accs_ma = []
        print(name_of_plot)
        log = 'Loss: {:.4f}, Epoch: {:03d}, Train acc micro: {:.4f}, Test acc micro: {:.4f},Train acc macro: {:.4f}, Test acc macro: {:.4f}'
         
        for epoch in range(100):
                    print(epoch)
                    loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch,self.loss)
                    losses.append(loss.detach().cpu())
                    [train_acc_mi, test_acc_mi,val_acc_mi],[train_acc_ma, test_acc_ma,val_acc_ma] = self.test(model,self.data,'logistic regression',c)
                    train_accs_mi.append(train_acc_mi)
                    test_accs_mi.append(test_acc_mi)
                    train_accs_ma.append(train_acc_ma)
                    test_accs_ma.append(test_acc_ma)
                    print(log.format(loss, epoch, train_acc_mi, test_acc_mi,train_acc_ma , test_acc_ma))
        
                    
                     #scheduler.step()
        print(log.format(loss, epoch, train_acc_mi, test_acc_mi,train_acc_ma , test_acc_ma))
        #print('Test acc on the last epoch micro ', test_acc_micro)
        #print(list(map(lambda x: float(x),losses)))
        plt.plot(losses)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
        plt.plot(test_accs_mi)
        plt.title(name_of_plot+' test f1 micro')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
                  
        plt.plot(test_accs_ma)
        plt.title(name_of_plot+' test f1 macro')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()

In [11]:
class MainOptuna(Main):
    def objective(self,trial):
        # Integer parameter
        hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
        out_layer = trial.suggest_categorical("out_layer", [32,64,128])
        dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
        size = trial.suggest_categorical("size of network, number of convs", [1,2,3])
        Conv = self.Conv
        learning_rate= trial.suggest_float("lr",5e-4,1e-2)
        c = trial.suggest_categorical("c",  [0.001, 0.01, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9, 10,20,30,40,50,60,70,80,90,100])
        # варьируем параметры
        loss_to_train={}
        for name in self.loss:
            
            if type(self.loss[name]) == list :
                if len(self.loss[name]) == 3:
                    var = trial.suggest_int(name,self.loss[name][0],self.loss[name][1],step=self.loss[name][2])
                    loss_to_train[name] = var
                elif len(self.loss[name]) == 2:
                    var_2 = trial.suggest_float(name,self.loss[name][0],self.loss[name][1])
                    loss_to_train[name] = var_2
                else:
                    var_3 = trial.suggest_categorical(name, self.loss[name])
                    loss_to_train[name] = var_3
            else:
                loss_to_train[name] = self.loss[name]
        Sampler =loss_to_train["Sampler"]
        model = Net(dataset = self.data,mode=self.mode,conv=Conv,loss_function=loss_to_train,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = size,dropout = dropout)
        train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)),node_idx=self.train_mask, sizes=[-1]*size)
       # train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        LossSampler = Sampler(self.datasetname,self.data,device=self.device,mask=self.train_mask,loss_info=loss_to_train)
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  

        classifier = "logistic regression" #trial.suggest_categorical("classifier", ["logistic regression", "catboost"])

        if classifier == "catboost":
            n_estimators = trial.suggest_int("n of estimators", 10,40,5)
            learning_rate_catboost = trial.suggest_float("lr_catboost",5e-4,1e-2)
            max_depth = trial.suggest_int("max_depth",1,10,2)
        else:
            n_estimators = -1
            learning_rate_catboost =-1
            max_depth = -1
        #training of the model
        for epoch in range(50):
            #print('hi',self.data.x.shape)
            loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch,loss_to_train)
            #self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)
        [train_acc_mi, test_acc_mi,val_acc_mi], [train_acc_ma, test_acc_ma,val_acc_ma] = self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth,c=c)
        trial.report( np.sqrt(val_acc_mi*val_acc_ma) ,epoch)
        return np.sqrt(val_acc_mi*val_acc_ma)

    
    def run(self,number_of_trials):

        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss,"+str(self.Conv)+" conv")
        study.optimize(self.objective,n_trials = number_of_trials)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))

In [12]:
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
#хорошо
DeepWalk = {"Name": "DeepWalk","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p":1,"q":1,"loss var": "Random Walks","flag_tosave":False,"Sampler" : SamplerRandomWalk } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p": [0.25, 0.50, 1, 2, 4] ,"q":[0.25, 0.50, 1, 2, 4], "loss var": "Random Walks","flag_tosave":False,"Sampler": SamplerRandomWalk}#то же самое 
LINE = {"Name": "LINE","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix} 

#плохо
HOPE_RPR = {"Name": "HOPE_RPR","C":"RPR","loss var": "Factorization","flag_tosave":True,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} #проверить
HOPE_Katz = {"Name": "HOPE_Katz","C":"Katz","loss var": "Factorization","flag_tosave":True,"betta": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} #проверить
HOPE_AA = {"Name": "HOPE_AdamicAdar","C":"AA","loss var": "Factorization","flag_tosave":True,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} 
HOPE_CN = {"Name": "HOPE_CommonNeighbors","C":"CN","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} 

LapEigen = {"Name": "LaplacianEigenMaps", "C":"Adj","loss var": "Laplacian EigenMaps","flag_tosave":True,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]}
VERSE_PPR =  {"Name": "VERSE_PPR","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerContextMatrix}

#под вопросом
VERSE_SR =  {"Name": "VERSE_SimRank","C": "SR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"Sampler":SamplerContextMatrix} 

Struc2Vec ={} #Implement
#в приципе можно найти нормальны результат через оптюну, но в основном плохо
GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization,"lmbda": [0.0,1.0]} 
APP ={"Name": "APP","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":True,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerAPP}
VERSE_Adj =  {"Name": "VERSE_Adj","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix} 
#CHAMELEON

In [17]:
 device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
loss=HOPE_CN
MO =MainOptuna('Cora','GCN', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

[I 2021-06-01 18:28:56,856] A new study created in memory with name: HOPE_CommonNeighbors loss,GCN conv
[I 2021-06-01 18:29:02,236] Trial 0 finished with value: 0.20629599946472696 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 2, 'lr': 0.0008564547982600537, 'c': 0.01, 'lmbda': 0.9219116541949283}. Best is trial 0 with value: 0.20629599946472696.
[I 2021-06-01 18:29:07,565] Trial 1 finished with value: 0.6839753593087904 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.00759037571430651, 'c': 50, 'lmbda': 0.10916755616959528}. Best is trial 1 with value: 0.6839753593087904.
[I 2021-06-01 18:29:12,812] Trial 2 finished with value: 0.22542098709711406 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 3, 'lr': 0.004879943274651615, 'c': 0.3, 'lmbda': 0.5009775212422715}. Best is trial 1 with value: 0.6839

[I 2021-06-01 18:31:20,381] Trial 26 finished with value: 0.21247896040771497 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.1, 'size of network, number of convs': 3, 'lr': 0.009381614222909892, 'c': 70, 'lmbda': 0.16975952640114617}. Best is trial 22 with value: 0.7520748411147121.
[I 2021-06-01 18:31:25,610] Trial 27 finished with value: 0.6552900436549327 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008437798362021099, 'c': 8, 'lmbda': 0.3949657698945761}. Best is trial 22 with value: 0.7520748411147121.
[I 2021-06-01 18:31:30,905] Trial 28 finished with value: 0.7020332724265429 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.009869102466443458, 'c': 9, 'lmbda': 0.03231338654821303}. Best is trial 22 with value: 0.7520748411147121.
[I 2021-06-01 18:31:36,383] Trial 29 finished with value: 0.6219097233302003 and parameter

[I 2021-06-01 18:33:38,836] Trial 52 finished with value: 0.7745225810897645 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009981126147980038, 'c': 50, 'lmbda': 0.16749861383914236}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:33:44,046] Trial 53 finished with value: 0.5126700044618119 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009900039753276553, 'c': 0.3, 'lmbda': 0.12641996690883112}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:33:49,510] Trial 54 finished with value: 0.7566184711160571 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.00999251244216572, 'c': 50, 'lmbda': 0.15414438367410238}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:33:54,996] Trial 55 finished with value: 0.7688751892478091 and para

[I 2021-06-01 18:35:58,983] Trial 78 finished with value: 0.6701625864617475 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.005343295373915275, 'c': 7, 'lmbda': 0.08828346335707181}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:36:04,144] Trial 79 finished with value: 0.2033262558005446 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.003225945417591315, 'c': 0.1, 'lmbda': 0.19287229073502457}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:36:09,596] Trial 80 finished with value: 0.7270287304945964 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.003454857643926192, 'c': 50, 'lmbda': 0.8267495942438308}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:36:15,039] Trial 81 finished with value: 0.7421700246

[I 2021-06-01 18:38:18,568] Trial 104 finished with value: 0.6331888195146602 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.005440856353773256, 'c': 4, 'lmbda': 0.10970734362287902}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:38:23,780] Trial 105 finished with value: 0.6866319215049699 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.1, 'size of network, number of convs': 1, 'lr': 0.005703497224710849, 'c': 10, 'lmbda': 0.19582078429015248}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:38:28,915] Trial 106 finished with value: 0.44023143914623103 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.004932251302802542, 'c': 0.3, 'lmbda': 0.03594285052375447}. Best is trial 52 with value: 0.7745225810897645.
[I 2021-06-01 18:38:34,367] Trial 107 finished with value: 0.7390081402524592 and 

[I 2021-06-01 18:40:39,847] Trial 130 finished with value: 0.7635597803714429 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009979985888926927, 'c': 100, 'lmbda': 0.06387106799118973}. Best is trial 126 with value: 0.7753454628892348.
[I 2021-06-01 18:40:45,398] Trial 131 finished with value: 0.7663269984561848 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009964679906482404, 'c': 100, 'lmbda': 0.06255399897052366}. Best is trial 126 with value: 0.7753454628892348.
[I 2021-06-01 18:40:50,902] Trial 132 finished with value: 0.7697248882124155 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009969965656717018, 'c': 100, 'lmbda': 0.060196401947411916}. Best is trial 126 with value: 0.7753454628892348.
[I 2021-06-01 18:40:56,492] Trial 133 finished with value: 0.7468590800991902 

[I 2021-06-01 18:43:03,794] Trial 156 finished with value: 0.7531316530669309 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.00836471330851591, 'c': 100, 'lmbda': 0.7266512898977895}. Best is trial 153 with value: 0.7840962413933994.
[I 2021-06-01 18:43:09,343] Trial 157 finished with value: 0.7503674901634231 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008910512546251546, 'c': 70, 'lmbda': 0.02851922340406697}. Best is trial 153 with value: 0.7840962413933994.
[I 2021-06-01 18:43:14,727] Trial 158 finished with value: 0.7069718057779029 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008633547506973005, 'c': 8, 'lmbda': 0.0005555538184176517}. Best is trial 153 with value: 0.7840962413933994.
[I 2021-06-01 18:43:20,434] Trial 159 finished with value: 0.7432645268021311 and 

[I 2021-06-01 18:45:27,124] Trial 182 finished with value: 0.5720296884667239 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009585743609611922, 'c': 0.4, 'lmbda': 0.03161934343944954}. Best is trial 153 with value: 0.7840962413933994.
[I 2021-06-01 18:45:32,959] Trial 183 finished with value: 0.7453380912843794 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.009970474693130029, 'c': 100, 'lmbda': 0.05173715395967064}. Best is trial 153 with value: 0.7840962413933994.


KeyboardInterrupt: 

In [14]:
 device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
loss=APP
MO =MainOptuna('Cora','GAT', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

[I 2021-06-01 18:10:15,147] A new study created in memory with name: APP loss,GAT conv
[I 2021-06-01 18:10:16,615] Trial 0 finished with value: 0.18782236460086427 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 2, 'lr': 0.007359858694733549, 'c': 0.1, 'num_negative_samples': 1, 'alpha': 0.6}. Best is trial 0 with value: 0.18782236460086427.
[I 2021-06-01 18:10:17,334] Trial 1 finished with value: 0.1601975513293414 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.0065927578453056235, 'c': 1, 'num_negative_samples': 16, 'alpha': 0.6}. Best is trial 0 with value: 0.18782236460086427.
[I 2021-06-01 18:10:18,076] Trial 2 finished with value: 0.23376903507951954 and parameters: {'hidden_layer': 256, 'out_layer': 64, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.001776470242817122, 'c': 6, 'num_negative_samples': 1, 'alpha': 0.9}. B

[I 2021-06-01 18:10:39,298] Trial 25 finished with value: 0.34965177917720147 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.008715918948101187, 'c': 0.4, 'num_negative_samples': 11, 'alpha': 0.9}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:10:39,999] Trial 26 finished with value: 0.59025046044078 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.1, 'size of network, number of convs': 1, 'lr': 0.00991320291397252, 'c': 0.7, 'num_negative_samples': 16, 'alpha': 0.2}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:10:41,117] Trial 27 finished with value: 0.7769025086928815 and parameters: {'hidden_layer': 128, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.007758595371285353, 'c': 100, 'num_negative_samples': 21, 'alpha': 0.1}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:10:42,134] Trial 28 finished with 

[I 2021-06-01 18:11:00,982] Trial 50 finished with value: 0.5906078285988741 and parameters: {'hidden_layer': 32, 'out_layer': 32, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.006508059726059234, 'c': 40, 'num_negative_samples': 21, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:02,065] Trial 51 finished with value: 0.761927518579239 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.006888280801103386, 'c': 20, 'num_negative_samples': 21, 'alpha': 0.1}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:02,723] Trial 52 finished with value: 0.7831670889458853 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.0, 'size of network, number of convs': 1, 'lr': 0.006749323853227683, 'c': 20, 'num_negative_samples': 21, 'alpha': 0.5}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:03,587] Trial 53 finished with val

[I 2021-06-01 18:11:25,173] Trial 75 finished with value: 0.8010051637685532 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008421173511992874, 'c': 50, 'num_negative_samples': 11, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:26,600] Trial 76 finished with value: 0.687878317128491 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 2, 'lr': 0.008411429844668217, 'c': 50, 'num_negative_samples': 11, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:27,866] Trial 77 finished with value: 0.800452693352377 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008755665751653681, 'c': 50, 'num_negative_samples': 11, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:29,068] Trial 78 finished with 

[I 2021-06-01 18:11:52,432] Trial 100 finished with value: 0.8004833780624571 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.009577628347886995, 'c': 50, 'num_negative_samples': 6, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:53,648] Trial 101 finished with value: 0.7912960381667579 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.00962802028958948, 'c': 50, 'num_negative_samples': 6, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:54,861] Trial 102 finished with value: 0.7822541350957506 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.007111389497180684, 'c': 50, 'num_negative_samples': 6, 'alpha': 0.3}. Best is trial 12 with value: 0.8058967193641101.
[I 2021-06-01 18:11:55,523] Trial 103 finished with v

...

[I 2021-06-01 18:16:48,701] Trial 400 finished with value: 0.6719077325571643 and parameters: {'hidden_layer': 64, 'out_layer': 32, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008211594045231354, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:16:49,515] Trial 401 finished with value: 0.7954775491738684 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008495474626004348, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:16:50,381] Trial 402 finished with value: 0.8023084717248081 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008756465448601522, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:16:51,221] Trial 403 finished 

[I 2021-06-01 18:17:10,935] Trial 425 finished with value: 0.620933294661567 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 3, 'lr': 0.008344043453236472, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.7}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:12,148] Trial 426 finished with value: 0.7801534212133556 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.007601293187042362, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.6}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:12,643] Trial 427 finished with value: 0.348906139140094 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.00811347272022226, 'c': 0.4, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:13,902] Trial 428 finished w

[I 2021-06-01 18:17:33,781] Trial 450 finished with value: 0.8006847322222913 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.007958446136825599, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:34,570] Trial 451 finished with value: 0.7974643763633118 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.0086526890043256, 'c': 70, 'num_negative_samples': 11, 'alpha': 0.8}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:35,251] Trial 452 finished with value: 0.8008014741140378 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.006626559775584429, 'c': 40, 'num_negative_samples': 11, 'alpha': 0.7}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:36,535] Trial 453 finished w

[I 2021-06-01 18:17:57,030] Trial 475 finished with value: 0.8019998728907517 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.0098848013974618, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.9}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:57,778] Trial 476 finished with value: 0.7909698590057631 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008080843790603151, 'c': 80, 'num_negative_samples': 11, 'alpha': 0.8}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:58,875] Trial 477 finished with value: 0.5292111530740461 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 2, 'lr': 0.009696625520404137, 'c': 0.6, 'num_negative_samples': 11, 'alpha': 0.9}. Best is trial 389 with value: 0.8233451826367902.
[I 2021-06-01 18:17:59,563] Trial 478 finished 

Best trial:
 Value:  0.8233451826367902
 Params: 
 hidden_layer: 64
 out_layer: 128
 dropout: 0.5
 size of network, number of convs: 1
 lr: 0.008039132462742
 c: 80
 num_negative_samples: 11
 alpha: 0.7


In [15]:
 device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
loss=APP
MO =MainOptuna('Cora','SAGE', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

[I 2021-06-01 18:18:18,111] A new study created in memory with name: APP loss,SAGE conv
[I 2021-06-01 18:18:18,769] Trial 0 finished with value: 0.4206298841531089 and parameters: {'hidden_layer': 256, 'out_layer': 64, 'dropout': 0.4, 'size of network, number of convs': 2, 'lr': 0.0051522269726015, 'c': 5, 'num_negative_samples': 16, 'alpha': 0.8}. Best is trial 0 with value: 0.4206298841531089.
[I 2021-06-01 18:18:19,211] Trial 1 finished with value: 0.3180738688103332 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007478504542106579, 'c': 0.2, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 0 with value: 0.4206298841531089.
[I 2021-06-01 18:18:19,981] Trial 2 finished with value: 0.5714328118695438 and parameters: {'hidden_layer': 32, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008347057984538225, 'c': 3, 'num_negative_samples': 16, 'alpha': 0.1}. Best is trial 2 with v

[I 2021-06-01 18:18:40,972] Trial 25 finished with value: 0.15975805104625634 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.009378771984474845, 'c': 0.001, 'num_negative_samples': 1, 'alpha': 0.4}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:18:41,581] Trial 26 finished with value: 0.5962879455302182 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.007765294158045356, 'c': 0.7, 'num_negative_samples': 1, 'alpha': 0.4}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:18:42,829] Trial 27 finished with value: 0.7711531607169613 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.009294126669425695, 'c': 100, 'num_negative_samples': 1, 'alpha': 0.9}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:18:44,119] Trial 28 finished wit

[I 2021-06-01 18:19:02,063] Trial 50 finished with value: 0.46275516701720176 and parameters: {'hidden_layer': 64, 'out_layer': 32, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007598291827359009, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:02,936] Trial 51 finished with value: 0.7626783395589481 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007832489002445273, 'c': 100, 'num_negative_samples': 11, 'alpha': 0.5}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:03,826] Trial 52 finished with value: 0.7517171193579859 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.30000000000000004, 'size of network, number of convs': 1, 'lr': 0.00821671149547199, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:04,605] Trial 

[I 2021-06-01 18:19:24,928] Trial 75 finished with value: 0.7707032821399776 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.0077133559463177966, 'c': 100, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:25,652] Trial 76 finished with value: 0.2682090510427094 and parameters: {'hidden_layer': 256, 'out_layer': 64, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.007391896360790098, 'c': 0.9, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:26,680] Trial 77 finished with value: 0.7383459821196471 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.0076716224706311655, 'c': 10, 'num_negative_samples': 11, 'alpha': 0.1}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:27,636] Trial 78 finished w

[I 2021-06-01 18:19:47,919] Trial 100 finished with value: 0.7532423736506935 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007553423097385504, 'c': 70, 'num_negative_samples': 1, 'alpha': 0.2}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:48,849] Trial 101 finished with value: 0.7678961644086207 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007358475292399903, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:49,730] Trial 102 finished with value: 0.7521048569756704 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.00799994251042393, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 23 with value: 0.7871943689839175.
[I 2021-06-01 18:19:50,671] Trial 103 finished w

...

[I 2021-06-01 18:24:54,562] Trial 400 finished with value: 0.7780082250746595 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.008812737988932071, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:24:56,180] Trial 401 finished with value: 0.6411938890448192 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 2, 'lr': 0.007143721137750711, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.3}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:24:57,151] Trial 402 finished with value: 0.7102090884552326 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.008990960368199505, 'c': 10, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:24:58,096] Trial 403 finish

[I 2021-06-01 18:25:21,286] Trial 425 finished with value: 0.5594149751704969 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.00815670319109493, 'c': 0.3, 'num_negative_samples': 6, 'alpha': 0.9}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:22,164] Trial 426 finished with value: 0.7470441954357315 and parameters: {'hidden_layer': 128, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.007265223969733045, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.7}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:23,002] Trial 427 finished with value: 0.7822448030420917 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.008876484378131507, 'c': 70, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:23,454] Trial 428 finished

[I 2021-06-01 18:25:41,652] Trial 450 finished with value: 0.7308246424359301 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.5, 'size of network, number of convs': 1, 'lr': 0.00852453939384248, 'c': 7, 'num_negative_samples': 6, 'alpha': 0.6}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:42,580] Trial 451 finished with value: 0.7669199093034467 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.4, 'size of network, number of convs': 1, 'lr': 0.00791728236121762, 'c': 100, 'num_negative_samples': 6, 'alpha': 0.5}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:43,261] Trial 452 finished with value: 0.6451534108143904 and parameters: {'hidden_layer': 64, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.007351017363110395, 'c': 3, 'num_negative_samples': 6, 'alpha': 0.4}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:25:44,152] Trial 453 finished with 

[I 2021-06-01 18:26:08,399] Trial 475 finished with value: 0.7458999333859555 and parameters: {'hidden_layer': 32, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.0086198996552949, 'c': 30, 'num_negative_samples': 6, 'alpha': 0.6}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:26:09,341] Trial 476 finished with value: 0.6778562297352868 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.008223258139433564, 'c': 4, 'num_negative_samples': 6, 'alpha': 0.6}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:26:10,229] Trial 477 finished with value: 0.5015648160097473 and parameters: {'hidden_layer': 256, 'out_layer': 128, 'dropout': 0.2, 'size of network, number of convs': 1, 'lr': 0.005555967835280485, 'c': 0.4, 'num_negative_samples': 6, 'alpha': 0.6}. Best is trial 162 with value: 0.7916208460702236.
[I 2021-06-01 18:26:10,814] Trial 478 finished wi

Best trial:
 Value:  0.7916208460702236
 Params: 
 hidden_layer: 64
 out_layer: 128
 dropout: 0.4
 size of network, number of convs: 1
 lr: 0.007788545472514005
 c: 100
 num_negative_samples: 6
 alpha: 0.9


In [16]:
mnpppppppppppppp

NameError: name 'mnpppppppppppppp' is not defined

In [ ]:
 device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
loss=Node2Vec
MO =MainOptuna('C','GAT', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

In [ ]:
 device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
loss=Node2Vec
MO =MainOptuna('Citeseer','SAGE', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=500)

In [ ]:
9 ru9

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')
walk_length= 20
walks_per_node= 5 
num_negative_samples= 11 
context_size= 10
p=1
q=1
DeepWalk = {"Name": "DeepWalk","walk_length":walk_length,"walks_per_node":walks_per_node,"num_negative_samples":num_negative_samples,"context_size" :context_size,"p":1,"q":1,"loss var": "Random Walks","flag_tosave":False,"Sampler" : SamplerRandomWalk } #Проблемы с памятью после того, как увеличила количество тренировочных данных

loss=DeepWalk
MO = Main('Cora','GCN', device, loss , 'unsupervised',dataset,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(hidden_layer=128,out_layer=128,dropout=0.4,size=2,learning_rate=0.005,c=80)